Gleiches Szenario wie bei ErstesSzenario aber hier haben wir 6 Kamera Sensoren installiert auf das Auto.

In [ ]:
%pip install carla

In [ ]:
import carla
import math
import random
import time
import numpy as np
import cv2
import open3d as o3d 
from matplotlib import cm 

In [ ]:
# Connect the client and set up bp library and spawn points
client = carla.Client('localhost', 2000)
world = client.get_world()
bp_lib = world.get_blueprint_library()
spawn_points = world.get_map().get_spawn_points()

In [ ]:
#Load a World
client.load_world('Town05')

In [ ]:
#Add the ego_vehicle
vehicle_bp_benz= bp_lib.find('vehicle.mercedes.coupe_2020')
vehicle_bp_benz.set_attribute('color', '255,255,255')
ego_vehicle = world.spawn_actor(vehicle_bp_benz, spawn_points[0])

In [ ]:
#Move spectator behind to view
spectator = world.get_spectator()
transform = carla.Transform(ego_vehicle.get_transform().transform(carla.Location(x=-6, z=2.0)), ego_vehicle.get_transform().rotation)
spectator.set_transform(transform)

In [ ]:
#Security Auto vor dem ego_vehicle
vehicle_bp_benz.set_attribute('color', '0,0,0')
transform = carla.Transform(ego_vehicle.get_transform().transform(carla.Location(x=6, z=0.5)), ego_vehicle.get_transform().rotation)
security_vehicle_front = world.spawn_actor(vehicle_bp_benz, transform)

In [ ]:
#Security Auto Hinter dem ego_vehicle
vehicle_bp_benz.set_attribute('color', '0,0,0')
transform = carla.Transform(ego_vehicle.get_transform().transform(carla.Location(x=-6, z=0.5)), ego_vehicle.get_transform().rotation)
security_vehicle_behind = world.spawn_actor(vehicle_bp_benz, transform)

In [ ]:
#Wir erstellen 30 zufällige Autos
vehicle_bp = bp_lib.filter("*vehicle*")

for i in range(30):

    npc_vehicle = world.try_spawn_actor(random.choice(vehicle_bp), random.choice(spawn_points))

In [ ]:
# Set initial camera translation
camera_init_trans = carla.Transform(carla.Location(z=2))

# Add one of each type of camera
camera_bp = bp_lib.find('sensor.camera.rgb') 
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=ego_vehicle)

sem_camera_bp = bp_lib.find('sensor.camera.semantic_segmentation') 
sem_camera = world.spawn_actor(sem_camera_bp, camera_init_trans, attach_to=ego_vehicle)

inst_camera_bp = bp_lib.find('sensor.camera.instance_segmentation') 
inst_camera = world.spawn_actor(inst_camera_bp, camera_init_trans, attach_to=ego_vehicle)

depth_camera_bp = bp_lib.find('sensor.camera.depth') 
depth_camera = world.spawn_actor(depth_camera_bp, camera_init_trans, attach_to=ego_vehicle)

dvs_camera_bp = bp_lib.find('sensor.camera.dvs') 
dvs_camera = world.spawn_actor(dvs_camera_bp, camera_init_trans, attach_to=ego_vehicle)

opt_camera_bp = bp_lib.find('sensor.camera.optical_flow') 
opt_camera = world.spawn_actor(opt_camera_bp, camera_init_trans, attach_to=ego_vehicle)

In [ ]:
# Define respective callbacks
def rgb_callback(image, data_dict):
    data_dict['rgb_image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))
    
def sem_callback(image, data_dict):
    image.convert(carla.ColorConverter.CityScapesPalette)
    data_dict['sem_image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

def inst_callback(image, data_dict):
    data_dict['inst_image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

def depth_callback(image, data_dict):
    image.convert(carla.ColorConverter.LogarithmicDepth)
    data_dict['depth_image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))
    
def opt_callback(data, data_dict):
    image = data.get_color_coded_flow()
    img = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))
    img[:,:,3] = 255
    data_dict['opt_image'] = img
    
def dvs_callback(data, data_dict):
    dvs_events = np.frombuffer(data.raw_data, dtype=np.dtype([
                ('x', np.uint16), ('y', np.uint16), ('t', np.int64), ('pol', np.bool)]))
    data_dict['dvs_image'] = np.zeros((data.height, data.width, 4), dtype=np.uint8)
    dvs_img = np.zeros((data.height, data.width, 3), dtype=np.uint8)
    dvs_img[dvs_events[:]['y'], dvs_events[:]['x'], dvs_events[:]['pol'] * 2] = 255
    data_dict['dvs_image'][:,:,0:3] = dvs_img
    

In [ ]:
# Set traffic in motion
for vehicle in world.get_actors().filter('*vehicle*'): 
    vehicle.set_autopilot(True) 

In [ ]:
image_w = camera_bp.get_attribute("image_size_x").as_int()
image_h = camera_bp.get_attribute("image_size_y").as_int()


sensor_data = {'rgb_image': np.zeros((image_h, image_w, 4)),
               'sem_image': np.zeros((image_h, image_w, 4)),
               'depth_image': np.zeros((image_h, image_w, 4)),
               'dvs_image': np.zeros((image_h, image_w, 4)),
               'opt_image': np.zeros((image_h, image_w, 4)), 
               'inst_image': np.zeros((image_h, image_w, 4))}

# Set sensors recording
camera.listen(lambda image: rgb_callback(image, sensor_data))
sem_camera.listen(lambda image: sem_callback(image, sensor_data))
inst_camera.listen(lambda image: inst_callback(image, sensor_data))
depth_camera.listen(lambda image: depth_callback(image, sensor_data))
dvs_camera.listen(lambda image: dvs_callback(image, sensor_data))
opt_camera.listen(lambda image: opt_callback(image, sensor_data))


# OpenCV named window for display
cv2.namedWindow('RGB Camera', cv2.WINDOW_AUTOSIZE)
cv2.namedWindow('Sem Camera', cv2.WINDOW_AUTOSIZE)
cv2.namedWindow('Depth Camera', cv2.WINDOW_AUTOSIZE)
cv2.namedWindow('DVS Camera', cv2.WINDOW_AUTOSIZE)
cv2.namedWindow('OPT Camera', cv2.WINDOW_AUTOSIZE)
cv2.namedWindow('Inst Camera', cv2.WINDOW_AUTOSIZE)

# Dispaly with imshow
cv2.imshow('RGB Camera', sensor_data['rgb_image'])
cv2.imshow('Sem Camera', sensor_data['sem_image'])
cv2.imshow('Depth Camera', sensor_data['inst_image'])
cv2.imshow('DVS Camera', sensor_data['depth_image'])
cv2.imshow('OPT Camera', sensor_data['dvs_image'])
cv2.imshow('Inst Camera', sensor_data['opt_image'])

cv2.waitKey(1)


# Indefinite while loop
while True:
    cv2.imshow('RGB Camera', sensor_data['rgb_image'])
    cv2.imshow('Sem Camera', sensor_data['sem_image'])
    cv2.imshow('Depth Camera', sensor_data['inst_image'])
    cv2.imshow('DVS Camera', sensor_data['depth_image'])
    cv2.imshow('OPT Camera', sensor_data['dvs_image'])
    cv2.imshow('Inst Camera', sensor_data['opt_image'])

    # Break loop if user presses q
    if cv2.waitKey(1) == ord('q'):
        break

# Stop sensors and destroy OpenCV window
camera.stop()
sem_camera.stop()
inst_camera.stop()
depth_camera.stop()
dvs_camera.stop()
opt_camera.stop()
cv2.destroyAllWindows()
    
       
    
    
